### Memory management and Garbage Collection With GUI

In [7]:
from tkinter import *
#globals
memory_size = 64
g_size=0

#Linked list utilities
class free_node:
    def __init__(self,index,size):
        self.index = index
        self.size = size
        self.next = None
        self.mark = 0
        self.prev = None

#initially whole memory is free
free_head = free_node(0,memory_size)

def delete_node(itr):
    global free_head
    prev_node = itr.prev
    next_node = itr.next
    if prev_node == None:
        free_head = next_node
        if free_head != None:
            free_head.prev = None
        del itr
    else:
        prev_node.next = next_node
        if next_node != None:
            next_node.prev = prev_node
        del itr

def add_to_free(index,size):
    global free_head
    node = free_node(index,size)
    if free_head == None:
        free_head = node
    else:
        cur = free_head
        prev = None
        while(cur != None and cur.index<index):
            prev = cur
            cur = cur.next
        if prev == None:
            if free_head.index == node.index + node.size:
                free_head.index = node.index
                free_head.size = free_head.size + node.size
            else:
                node.next = free_head
                free_head.prev = node
                node.prev = None
                free_head = node
        elif cur == None:
            if node.index == prev.index+prev.size:
                prev.size = prev.size+node.size
            else:
                prev.next = node
                node.prev = prev
                node.next = None
        else:
            if prev.index+prev.size == node.index and node.index + node.size == cur.index:
                prev.size = prev.size + node.size+cur.size
                prev.next = cur.next
                del cur
                if cur.next != None:
                    cur.next.prev = prev
            elif prev.index+prev.size == node.index:
                prev.size = prev.size+node.size
            elif node.index + node.size == cur.index:
                cur.index = node.index
                cur.size = cur.size+node.size
            else:
                prev.next = node
                node.prev = prev
                node.next = cur
                cur.prev = node

#Simulating alloc and dealloc
def allocate_sim(memory,index,size,ptr_name):
    for i in range(index,index+size):
        memory[i].config(bg = 'black')
    memory[index]['text'] = ptr_name
def deallocate_sim(memory,index,size):
    memory[index]['text'] = ''
    for i in range(index,index+size):
        memory[i].config(bg = 'white')

def declare_garbage(memory,ptr_name,manager):
    for i in range(manager[ptr_name][0],manager[ptr_name][0]+manager[ptr_name][1]):
        memory[i].config(bg = 'red')
    memory[manager[ptr_name][0]]['text'] = 'GBG'

##ALLOCATING MEMORY USING FIRST FIT STRATEGY

def memory_alloc(memory,ptr_to_alloc,size,manager,status_label):
    global free_head
    ptr_name = ptr_to_alloc.get()
    size_to_alloc = size.get()
    #flagging errors if they are present
    try:
        ptr_name = str(ptr_name)
        size_to_alloc = int(size_to_alloc)
    except:
        pass
    else:
        if(size == 0):
            status_label['text'] = "0 sized memory\ncan not be\nallocated."
        elif(len(ptr_name) == 0):
            status_label['text'] = "Variable name\ncan\'t be a\nempty string."
        elif(len(ptr_name)<=50):
            itr = free_head
            found = False
            while itr != None and (not found):
                if itr.size >= size_to_alloc:
                    found = True
                else:
                    itr = itr.next
            if found:
                if ptr_name in manager:
                    declare_garbage(memory,ptr_name,manager) ##if ptr_name is reassigned without freeing the
                    #memory it was already assigned with,it becomes garbage
                    del manager[ptr_name]
                if itr.size == size_to_alloc:
                    itr.mark=1
                    allocate_sim(memory, itr.index, size_to_alloc,ptr_name)
                    manager[ptr_name] = [itr.index,size_to_alloc]
                    delete_node(itr)
                else:
                    itr.mark=1
                    allocate_sim(memory,itr.index,size_to_alloc,ptr_name)
                    manager[ptr_name] = [itr.index,size_to_alloc]
                    itr.size = itr.size-size_to_alloc
                    itr.index = itr.index + size_to_alloc
                status_label['text'] = "Status:\nMemory allocation\nsuccessful.."
            if not found:
                if ptr_name in manager:
                    declare_garbage(memory,ptr_name,manager) ##if ptr_name is reassigned without freeing the
                    #memory it was already assigned with,it becomes garbage
                    global g_size
                    g_size=g_size+manager[ptr_name][1]
                status_label['text'] = "Status:\nMemory allocation\nunsuccessful.."
       

def remove_red_blocks(memory,manager):
    for i in range(0,memory_size):
        if memory[i]['bg'] == 'red':
            memory[i]['text'] = ''
            memory[i].config(bg = 'white') 


def garbage_collector_ms(memory,manager):
    #free all the 0 marked nodes
    itr = free_head
    while itr != None:
        if itr.mark == 0:
            delete_node(itr)
        itr = itr.next
    add_to_free(0,g_size)
    #free all the garbage blocks
    for i in manager:
        if manager[i][0] == 'GBG':
            declare_garbage(memory,i,manager)
            del manager[i]
    remove_red_blocks(memory,manager)
    

def memory_dealloc(memory,manager,ptr_to_free,status_label):
    ptr=ptr_to_free
    if(type(ptr)==str):
        if ptr=="GBG":
            garbage_collector_ms(memory,manager)
            status_label['text'] = "Status:\nGarbage collection\nsuccessful.."
    else:
        ptr = ptr_to_free.get()
        if ptr not in manager:
            status_label['text'] = "Entered pointer\ndoes not\nexist.."
        else:
            deallocate_sim(memory,manager[ptr][0],manager[ptr][1])
            add_to_free(manager[ptr][0],manager[ptr][1])
            del manager[ptr]
            status_label['text'] = "Status:\nMemory Deallocated\nSuccessfully.."
        

if __name__ == "__main__":
    
    
    root = Tk()
    root.title('Heap management')
    root.configure(background = 'orange')
    root.geometry('900x900')
    manager = dict()
    #for memory simulation
    memory = []
    for i in range(100):
        memory.append(Button(root,text = "",state = DISABLED,bg = 'white'))
        
    x_off = 0.20
    y_off = 0.30

    for i in range(10):
        for j in range(10):
            memory[i*8+j].place(relx = x_off+(1/8)*j*0.60,rely = y_off+(1/8)*i*0.60,relheight = 1/8*0.60,relwidth = 1/8*0.60)

    #for messages
    message = "\n**U can ENTER GBG in ptr to free or\n press garbage colllector button to trigger garbage collector"
             
    message_label = Label(root,text = message,font = "Helvetica 8",bg = "orange")
    message_label.place(relx = 0,rely = 0.9,relheight = 0.1,relwidth = 1)
    status_label = Label(root,text = "",font = "Helvetica 10",bg = "orange")
    status_label.place(relx = 0.80,rely = 0.30,relwidth = 0.20,relheight = 0.60)

    #user helper grids
    back_gr = Label(root,text = '',bg = 'blue')
    back_gr.place(relx = 0,rely = 0.375,relwidth = 0.18,relheight = 0.525)
    mean = Label(root,text = "Legend",font = "italic",bg = 'blue')
    mean.place(relx = 0,rely = 0.375,relheight = 0.075,relwidth = 0.18)
    white_button = Button(root,text ='',state = DISABLED)
    white_button.place(relx = 0.05,rely = 0.45,relwidth = 0.075,relheight = 0.075)
    white_means = Label(root,text = "Free Cell",font = "Helvetica 10 bold",bg = 'blue')
    white_means.place(relx = 0,rely = 0.525,relwidth = 0.18,relheight = 0.075)
    black_button = Button(root, text='', state=DISABLED,bg = "black")
    black_button.place(relx=0.05, rely=0.60, relwidth=0.075, relheight=0.075)
    black_means = Label(root, text="Allocated Cell", font="Helvetica 10 bold",bg = 'blue')
    black_means.place(relx=0, rely=0.675, relwidth=0.18, relheight=0.075)
    red_button = Button(root,text = '',bg = 'red',state = DISABLED)
    red_button.place(relx = 0.05,rely = 0.75,relwidth=0.075, relheight=0.075)
    red_label = Label(root,text = "GBG:\nGarbage cell",font = "Helvetica 10 bold",bg = "blue")
    red_label.place(relx = 0,rely = 0.825,relwidth = 0.18,relheight = 0.075)


    #for allocation simulation
    l_alloc = Label(root,text = "Allocate a memory",font = "Times 20 italic bold",bg ="orange")
    l_alloc.place(relx = 0,rely = 0,relheight = 0.075,relwidth = 0.40)
    ptr_ask_alloc = Label(root, text="ptr name :", font="Times 20 italic bold", bg="orange")
    ptr_ask_alloc.place(relx=0, rely=0.075, relheight=0.075, relwidth=0.20)
    ptr_to_alloc = Entry(root,font = "Times 20 italic bold",justify = CENTER)
    ptr_to_alloc.place(relx = 0.20,rely = 0.075,relheight = 0.075,relwidth = 0.20)
    size_ask = Label(root, text="Size to allocate :", font="Times 20 italic bold", bg="orange")
    size_ask.place(relx=0, rely=0.15,relheight=0.075, relwidth=0.20)
    size = Entry(root, font="Times 20 italic bold", justify=CENTER)
    size.place(relx=0.20, rely=0.15, relheight=0.075, relwidth=0.20)

    #imp button
    submit_alloc = Button(root,text = "Submit",font = "Times 20 italic bold",command = lambda : memory_alloc(memory,ptr_to_alloc,size,manager,status_label))


    submit_alloc.place(relx = 0.20,rely = 0.225+0.0112,relheight = 0.05,relwidth = 0.20)

    #for deallocation(freeing memory) simulation
    l_free = Label(root,text = "Free the memory",font = "Times 20 italic bold",bg = 'orange')
    l_free.place(relx = 0.60,rely = 0,relheight = 0.075,relwidth = 0.40)
    ptr_ask_free = Label(root,text = "ptr name :",font = "Times 20 italic bold",bg = 'orange')
    ptr_ask_free.place(relx=0.60, rely=0.075, relheight=0.075, relwidth=0.20)
    ptr_to_free = Entry(root,font = "Times 20 italic bold",justify = CENTER)
    ptr_to_free.place(relx = 0.80,rely = 0.075, relheight = 0.075,relwidth = 0.20)

    #imp button
    submit_free = Button(text = "Submit",font = "Times 20 italic bold",command = lambda:memory_dealloc(memory,manager,ptr_to_free,status_label))
    submit_free.place(relx = 0.80,rely = 0.15+0.0125,relheight = 0.05,relwidth = 0.20)


    #for garbage collection simulation
    l_gc= Label(root,text = "Garbage collection :",font = "Times 20 italic bold",bg = 'orange')
    l_gc.place(relx = 0.70,rely = 0.225,relheight = 0.05,relwidth = 0.40)
    gc_button = Button(root,text = "Submit",font = "Times 20 italic bold",command = lambda : memory_dealloc(memory,manager,"GBG",status_label))
    gc_button.place(relx = 0.80,rely = 0.225+0.0500,relheight = 0.05,relwidth = 0.20)
    
    root.mainloop()


## Memory management and Garbage Collection With reference counting

In [9]:
class node:
    def __init__(self, num):
        #instance fields found by C++ to Python Converter:
        self.data = 0
        self.count = 0
        self.adjacent1 = None
        self.adjacent2 = None
        self.adjacent3 = None

        self.data = num
        self.adjacent1 = None
        self.adjacent2 = None
        self.adjacent3 = None
        self.count = 0 #3 connections are present
class root_tag:

    def __init__(self):
        #instance fields found by C++ to Python Converter:
        self.pointer = None

class Globals:
    heap = [None for _ in range(8)]
    #
    #root1->5 7 3
    #       |/|/|
    #root2->1 8 | 
    #       |\| |
    #       | |\|
    #       |\| |
    #       2 9 10
    #
    @staticmethod
    def initialize(root1, root2):
        temp = node(5)
        Globals.heap[0] = temp
        temp = node(1)
        Globals.heap[1] = temp
        temp = node(2)
        Globals.heap[2] = temp
        temp = node(9)
        Globals.heap[3] = temp
        temp = node(10)
        Globals.heap[4] = temp
        temp = node(7)
        Globals.heap[5] = temp
        temp = node(8)
        Globals.heap[6] = temp
        temp = node(3)
        Globals.heap[7] = temp
        temp = None
        #now all nodes are in the 'heap',to create connections now
        #while making connections, update reference counts
        root1.pointer = Globals.heap[0] #root1->5
        Globals.heap[0].count+=1
        Globals.heap[0].adjacent1 = Globals.heap[1] #5->1
        Globals.heap[1].count+=1
        root2.pointer = Globals.heap[1] #root2->1
        Globals.heap[1].count+=1
        Globals.heap[1].adjacent1 = Globals.heap[2] #1->2
        Globals.heap[2].count+=1
        Globals.heap[1].adjacent2 = Globals.heap[3] #1->9
        Globals.heap[3].count+=1
        Globals.heap[1].adjacent3 = Globals.heap[4] #1->10
        Globals.heap[4].count+=1
        Globals.heap[5].adjacent1 = Globals.heap[1] #7->1
        Globals.heap[1].count+=1
        Globals.heap[5].adjacent2 = Globals.heap[6] #7->8
        Globals.heap[6].count+=1
        Globals.heap[6].adjacent1 = Globals.heap[4] #8->10
        Globals.heap[4].count+=1
        Globals.heap[7].adjacent1 = Globals.heap[6] #3->8
        Globals.heap[6].count+=1
        Globals.heap[7].adjacent2 = Globals.heap[4] #3->10
        Globals.heap[4].count+=1
        #connections done
    @staticmethod
    def print_node(node):
        if node is None:
            return
        print(" ", end = '')
        print(node.data, end = '')
        print("(rfc=", end = '')
        print(node.count, end = '')
        print(")", end = '')
        if node.adjacent1 is not None or node.adjacent2 is not None or node.adjacent3 is not None:
            print("-{", end = '')
            Globals.print_node(node.adjacent1)
            Globals.print_node(node.adjacent2)
            Globals.print_node(node.adjacent3)
            print(" }", end = '')
    @staticmethod
    def print_heap(arr):
        for i in range(0, 8):
            if arr[i] is not None:
                Globals.print_node(arr[i])
                print("\n", end = '')
    @staticmethod
    def print_useful_heap(value):
        print("root->", end = '')
        Globals.print_node(value.pointer)
        print("\n", end = '')
    @staticmethod
    def garbage_collector_rf(arr):
        flag = 0
        for i in range(0, 8):
            if arr[i] is not None:
                if arr[i].count == 0:
                    if arr[i].adjacent1 is not None:
                        arr[i].adjacent1.count-=1
                        arr[i].adjacent1 = None
                    if arr[i].adjacent2 is not None:
                        arr[i].adjacent2.count-=1
                        arr[i].adjacent2 = None
                    if arr[i].adjacent3 is not None:
                        arr[i].adjacent3.count-=1
                        arr[i].adjacent3 = None
                    arr[i] = None
                    arr[i] = None
                    flag = 1
        if flag != 0: #rfc is changed so gc again
            Globals.garbage_collector_rf(arr)

A = root_tag()
B = root_tag()
Globals.initialize(A, B)
print(" Simulation of reference counting garbage collector\n", end = '')
print("legend: -> indicates connection and {} indicate all the elements connected to the element\n", end = '')
print("the full heap is: ", end = '')
print("\n", end = '')
Globals.print_heap(Globals.heap)
print("-----------------", end = '')
print("\n", end = '')
print("Heap connected to the roots is: ", end = '')
print("\n", end = '')
Globals.print_useful_heap(A)
Globals.print_useful_heap(B)
print("-----------------", end = '')
print("\n", end = '')
Globals.garbage_collector_rf(Globals.heap)
print("the garbage collector triggered,full heap disaplaying\n", end = '')
Globals.print_heap(Globals.heap)
print("matche heap connected to roots printed below\n", end = '')
Globals.print_useful_heap(A)
Globals.print_useful_heap(B)

 Simulation of reference counting garbage collector
legend: -> indicates connection and {} indicate all the elements connected to the element
the full heap is: 
 5(rfc=1)-{ 1(rfc=3)-{ 2(rfc=1) 9(rfc=1) 10(rfc=3) } }
 1(rfc=3)-{ 2(rfc=1) 9(rfc=1) 10(rfc=3) }
 2(rfc=1)
 9(rfc=1)
 10(rfc=3)
 7(rfc=0)-{ 1(rfc=3)-{ 2(rfc=1) 9(rfc=1) 10(rfc=3) } 8(rfc=2)-{ 10(rfc=3) } }
 8(rfc=2)-{ 10(rfc=3) }
 3(rfc=0)-{ 8(rfc=2)-{ 10(rfc=3) } 10(rfc=3) }
-----------------
Heap connected to the roots is: 
root-> 5(rfc=1)-{ 1(rfc=3)-{ 2(rfc=1) 9(rfc=1) 10(rfc=3) } }
root-> 1(rfc=3)-{ 2(rfc=1) 9(rfc=1) 10(rfc=3) }
-----------------
the garbage collector triggered,full heap disaplaying
 5(rfc=1)-{ 1(rfc=2)-{ 2(rfc=1) 9(rfc=1) 10(rfc=1) } }
 1(rfc=2)-{ 2(rfc=1) 9(rfc=1) 10(rfc=1) }
 2(rfc=1)
 9(rfc=1)
 10(rfc=1)
matche heap connected to roots printed below
root-> 5(rfc=1)-{ 1(rfc=2)-{ 2(rfc=1) 9(rfc=1) 10(rfc=1) } }
root-> 1(rfc=2)-{ 2(rfc=1) 9(rfc=1) 10(rfc=1) }


## Memory management and Garbage Collection With Mark-sweep-method

In [10]:
class node:
    def __init__(self, num):
        #instance fields found by C++ to Python Converter:
        self.data = 0
        self.mark = False
        self.adjacent1 = None
        self.adjacent2 = None
        self.adjacent3 = None

        self.data = num
        self.adjacent1 = None
        self.adjacent2 = None
        self.adjacent3 = None
        Globals.mark=False #we know at most 3 connections are present
class root_tag:

    def __init__(self):
        #instance fields found by C++ to Python Converter:
        self.pointer = None


class Globals:
    heap = [None for _ in range(8)]
    #
    #root1->5 7 3
    #       |/|/|
    #root2->1 8 | 
    #       |\| |
    #       | |\|
    #       |\| |
    #       2 9 10
    #
    @staticmethod
    def initialize(root1, root2):
        temp = node(5)
        Globals.heap[0] = temp
        temp = node(1)
        Globals.heap[1] = temp
        temp = node(2)
        Globals.heap[2] = temp
        temp = node(9)
        Globals.heap[3] = temp
        temp = node(10)
        Globals.heap[4] = temp
        temp = node(7)
        Globals.heap[5] = temp
        temp = node(8)
        Globals.heap[6] = temp
        temp = node(3)
        Globals.heap[7] = temp
        temp = None
        #create connections now
        root1.pointer = Globals.heap[0] #root1->5
        Globals.heap[0].adjacent1 = Globals.heap[1] #5->1
        root2.pointer = Globals.heap[1] #root2->1
        Globals.heap[1].adjacent1 = Globals.heap[2] #1->2
        Globals.heap[1].adjacent2 = Globals.heap[3] #1->9
        Globals.heap[1].adjacent3 = Globals.heap[4] #1->10
        Globals.heap[5].adjacent1 = Globals.heap[1] #7->1
        Globals.heap[5].adjacent2 = Globals.heap[6] #7->8
        Globals.heap[6].adjacent1 = Globals.heap[4] #8->10
        Globals.heap[7].adjacent1 = Globals.heap[6] #3->8
        Globals.heap[7].adjacent2 = Globals.heap[4] #3->10
        #connections done
    @staticmethod
    def mark_node(ptr):
        head = ptr
        tail = None
        middle = None
        flag = 1
        while head is not None:
            if not head.mark: #if node we are on is unmarked, mark it
                head.mark = True
            if head.adjacent1 is not None and not head.adjacent1.mark: #if adjacent node to this is unmarked
                tail = middle
                middle = head
                head = head.adjacent1
            elif head.adjacent2 is not None and not head.adjacent2.mark: 
                tail = middle
                middle = head
                head = head.adjacent2
            elif head.adjacent3 is not None and not head.adjacent3.mark:
                tail = middle
                middle = head
                head = head.adjacent3
            else:
                head = middle
                middle = tail
                tail = None

    @staticmethod
    def marker(value):
        Globals.mark_node(value.pointer)
    @staticmethod
    def sweep(arr):
        for i in range(0, 8):
            if arr[i] is not None:
                if not arr[i].mark:
                    #abandone the node
                    arr[i].adjacent1 = None
                    arr[i].adjacent2 = None
                    arr[i].adjacent3 = None
                    arr[i] = None
                    arr[i] = None
    @staticmethod
    def garbage_collector(r1, r2, hp):
        print("Mark phase started.......", end = '')
        print("\n", end = '')
        Globals.marker(r1)
        Globals.marker(r2)
        print("Marking  phase done", end = '')
        print("\n", end = '')
        print("Sweep phase started.......", end = '')
        print("\n", end = '')
        Globals.sweep(hp)
    @staticmethod
    def print_node(node):
        if node is None:
            return
        print(" ", end = '')
        print(node.data, end = '')
        if node.adjacent1 is not None or node.adjacent2 is not None or node.adjacent3 is not None:
            print("-{", end = '')
            Globals.print_node(node.adjacent1)
            Globals.print_node(node.adjacent2)
            Globals.print_node(node.adjacent3)
            print(" }", end = '')
    @staticmethod
    def print_heap(arr):
        for i in range(0, 8):
            if arr[i] is not None:
                Globals.print_node(arr[i])
                print("\n", end = '')
    @staticmethod
    def print_useful_heap(value):
        print("root->", end = '')
        Globals.print_node(value.pointer)
        print("\n", end = '')


A = root_tag()
B = root_tag()
Globals.initialize(A,B)
print("Simulation for mark sweep garbage collector\n", end = '')
print("legend: - indicates connection and {} indicate all the elements connected to the element\n", end = '')
print("Displaying full heap is: ", end = '')
print("\n", end = '')
Globals.print_heap(Globals.heap)
print("-----------------", end = '')
print("\n", end = '')
print("Heap connected to the roots is: ", end = '')
print("\n", end = '')
Globals.print_useful_heap(A)
Globals.print_useful_heap(B)
print("-----------------", end = '')
print("\n", end = '')
Globals.garbage_collector(A, B, Globals.heap)
print("gc triggered displaying heap\n", end = '')
Globals.print_heap(Globals.heap)
print("match heap connected to roots printed below\n", end = '')
Globals.print_useful_heap(A)
Globals.print_useful_heap(B)
print("-----------------", end = '')

Simulation for mark sweep garbage collector
legend: - indicates connection and {} indicate all the elements connected to the element
Displaying full heap is: 
 5-{ 1-{ 2 9 10 } }
 1-{ 2 9 10 }
 2
 9
 10
 7-{ 1-{ 2 9 10 } 8-{ 10 } }
 8-{ 10 }
 3-{ 8-{ 10 } 10 }
-----------------
Heap connected to the roots is: 
root-> 5-{ 1-{ 2 9 10 } }
root-> 1-{ 2 9 10 }
-----------------
Mark phase started.......
Marking  phase done
Sweep phase started.......
gc triggered displaying heap
 5-{ 1-{ 2 9 10 } }
 1-{ 2 9 10 }
 2
 9
 10
match heap connected to roots printed below
root-> 5-{ 1-{ 2 9 10 } }
root-> 1-{ 2 9 10 }
-----------------

# END